In [93]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import pickle
from tqdm.auto import tqdm

from sklearn.linear_model import LogisticRegression, RidgeCV, Ridge
from sklearn.ensemble import ExtraTreesClassifier, ExtraTreesRegressor, RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier, CatBoostRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from xgboost import XGBClassifier, XGBRegressor
import optuna

from library.Data_Preprocessing import Balance_Ratio
from library.Imbalance_Sampling import label_divide
from library.Aging_Score_Contour import score1
from library.AdaBoost import train_set, multiple_set, multiple_month, line_chart, cf_matrix, AUC, PR_curve, \
     multiple_curve, PR_matrix, best_threshold, all_optuna, optuna_history, AdaBoost_creator 
from library.XGBoost import XGBoost_creator
from library.LightGBM import LightGBM_creator
from library.CatBoost import CatBoost_creator
from library.Random_Forest import RandomForest_creator
from library.Extra_Trees import ExtraTrees_creator

os.chdir('C:/Users/user/Desktop/Darui_R08621110')  
os.getcwd()

ImportError: cannot import name 'AdaBoost_creator' from 'library.AdaBoost' (C:\Users\user\Desktop\Darui_R08621110\Master_Thesis\AUO aging test\Full_Version_Code\library\AdaBoost.py)

## 

In [84]:
def optimize_base(num_set, train_data, mode, TPE_multi, base_list, filename):
    
    best_param = {}
    month_list = train_data.keys()
    
    for i in tqdm(month_list):
        
        best_param[f'{i}'] = {}
        if 'XGBoost' in base_list:
            best_param[f'{i}'][f'XGBoost'], _ = all_optuna(num_set = num_set, 
                                                           all_data = train_data[f'{i}'], 
                                                           mode = mode, 
                                                           TPE_multi = TPE_multi, 
                                                           n_iter = 1,
                                                           filename = f'{filename}_XGBoost',
                                                           creator = XGBoost_creator
                                        )

        if 'LightGBM' in base_list:
            best_param[f'{i}'][f'LightGBM'], _ = all_optuna(num_set = num_set, 
                                                            all_data = train_data[f'{i}'], 
                                                            mode = mode, 
                                                            TPE_multi = TPE_multi, 
                                                            n_iter = 1,
                                                            filename = f'{filename}_LightGBM',
                                                            creator = LightGBM_creator
                                        )
        
        if 'CatBoost' in base_list:
            best_param[f'{i}'][f'CatBoost'], _ = all_optuna(num_set = num_set, 
                                                            all_data = train_data[f'{i}'], 
                                                            mode = mode, 
                                                            TPE_multi = TPE_multi, 
                                                            n_iter = 1,
                                                            filename = f'{filename}_CatBoost',
                                                            creator = CatBoost_creator
                                        )
            
        if 'RandomForest' in base_list:
            best_param[f'{i}'][f'RandomForest'], _ = all_optuna(num_set = num_set, 
                                                            all_data = train_data[f'{i}'], 
                                                            mode = mode, 
                                                            TPE_multi = TPE_multi, 
                                                            n_iter = 1,
                                                            filename = f'{filename}_RandomForest',
                                                            creator = RandomForest_creator
                                        )

        if 'ExtraTrees' in base_list:
            best_param[f'{i}'][f'ExtraTrees'], _ = all_optuna(num_set = num_set, 
                                                            all_data = train_data[f'{i}'], 
                                                            mode = mode, 
                                                            TPE_multi = TPE_multi, 
                                                            n_iter = 1,
                                                            filename = f'{filename}_ExtraTrees',
                                                            creator = ExtraTrees_creator
                                        )
            
    return best_param



base_param = optimize_base(num_set = 10, 
                           train_data = data_dict, 
                           mode = 'C', 
                           TPE_multi = False, 
                           base_list = ['XGBoost', 'LightGBM'], 
                           filename = 'runhist_array_m2m5_4criteria')

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

[I 2021-10-09 16:45:32,954] A new study created in memory with name: no-name-b32bbe13-0b56-40f7-9dd8-097f5e936a9b


Dataset0 :


C:\Users\user\anaconda3\envs\aging\lib\site-packages\optuna\progress_bar.py:47: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


...

Precision: 0.9809885931558935 
Recall: 0.9735849056603774 
Aging Rate: 0.5009523809523809
[I 2021-10-09 16:46:12,526] Trial 0 finished with value: 0.930579752352036 and parameters: {'objective': 'binary', 'boosting_type': 'gbdt', 'n_estimators': 300, 'subsample': 0.9, 'num_leaves': 40, 'min_child_samples': 12, 'max_depth': 11, 'learning_rate': 0.125, 'reg_alpha': 0.11389736350898758, 'reg_lambda': 4.218037669155812}. Best is trial 0 with value: 0.930579752352036.
Sampler is TPESampler
Dataset9 :


  0%|          | 0/1 [00:00<?, ?it/s]

Precision: 0.92 
Recall: 0.7666666666666667 
Aging Rate: 0.08650519031141868
Precision: 1.0 
Recall: 0.8333333333333334 
Aging Rate: 0.06920415224913495
Precision:

C:\Users\user\anaconda3\envs\aging\lib\site-packages\pandas\core\computation\expressions.py:205: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  f"evaluating in Python space because the {repr(op_str)} "
C:\Users\user\anaconda3\envs\aging\lib\site-packages\pandas\core\computation\expressions.py:205: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  f"evaluating in Python space because the {repr(op_str)} "
C:\Users\user\anaconda3\envs\aging\lib\site-packages\pandas\core\computation\expressions.py:205: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  f"evaluating in Python space because the {repr(op_str)} "
C:\Users\user\anaconda3\envs\aging\lib\site-packages\pandas\core\computation\expressions.py:205: UserWarning: evaluating in Python space becau

 0.9473684210526315 
Recall: 0.75 
Aging Rate: 0.0657439446366782
[I 2021-10-09 16:46:12,730] Trial 0 finished with value: 0.7759515570934257 and parameters: {'objective': 'binary', 'boosting_type': 'goss', 'n_estimators': 100, 'subsample': 0.5, 'num_leaves': 35, 'min_child_samples': 15, 'max_depth': 7, 'learning_rate': 0.275, 'reg_alpha': 0.001431639527677854, 'reg_lambda': 0.028813276202748164}. Best is trial 0 with value: 0.7759515570934257.
Sampler is TPESampler


In [69]:
def stratified_data(train_data, cv):
    
    good = train_data[train_data.GB == 0]
    bad = train_data[train_data.GB == 1]
    good_index = random.sample(good.index.to_list(), k = len(good))
    bad_index = random.sample(bad.index.to_list(), k = len(bad))
    
    train_x_dict = {}
    train_y_dict = {}
    valid_x_dict = {}
    valid_y_dict = {}
    for i in range(cv):
        
        if (i+1) == cv:
            good_valid_index = good_index[int(np.floor((i/cv)*len(good))) : ]
            bad_valid_index = bad_index[int(np.floor((i/cv)*len(bad))) : ]
        else:
            good_valid_index = good_index[int(np.floor((i/cv)*len(good))) : int(np.floor(((i+1)/cv)*len(good)))]
            bad_valid_index = bad_index[int(np.floor((i/cv)*len(bad))) : int(np.floor(((i+1)/cv)*len(bad)))]
        good_train_index = [x for x in good_index if x not in good_valid_index]
        bad_train_index = [x for x in bad_index if x not in bad_valid_index]
        
        good_train = good.loc[good_train_index]
        good_valid = good.loc[good_valid_index]
        bad_train = bad.loc[bad_train_index]
        bad_valid = bad.loc[bad_valid_index]
        train = pd.concat([good_train, bad_train], axis = 0)
        valid = pd.concat([good_valid, bad_valid], axis = 0)
        train_x_dict[i], train_y_dict[i], valid_x_dict[i], valid_y_dict[i] = label_divide(train, valid, 
                                                                                              train_only = False)

    return train_x_dict, train_y_dict, valid_x_dict, valid_y_dict



a, b, c, d = stratified_data(aaa, cv = 5)

In [ ]:
def transform_test(train_data, test_data, base_param):
    
    month_list = base_param.keys()
    model_list = base_param[month_list[0]].keys()
    
    for i in range(num_set):
        
        train_x, train_y, test_x, test_y = label_divide(train_data[f'set{i}'], test_data, train_only = False)
        model_predict = pd.DataFrame()
        if mode == 'C':
            
            if 'XGBoost' in model_list:
                predict = XGBoostC(train_x, test_x, train_y, test_y, base_param[month_list[-1]]['XGBoost'])
                predict

In [87]:
'LightGBM' in base_param['m2'].keys()

True

In [103]:
kkk = pd.DataFrame(d[0])rename(columns = 'aa')
pd.concat([kkk, kkk], axis = 1)

,GB,GB
171,0,0
458,0,0
228,0,0
164,0,0
48,0,0
...,...,...
825,1,1
595,1,1
792,1,1
837,1,1


In [100]:
d[0]

171     0
458     0
228     0
164     0
48      0
       ..
825     1
595     1
792     1
837     1
1107    1
Name: GB, Length: 224, dtype: int64

## 

### loading training & testing data

In [5]:
### training data ### 
training_month = [2, 3, 4]

data_dict, trainset_x, trainset_y = multiple_month(training_month, num_set = 10, filename = 'dataset')

print('\nCombined training data:\n')
run_train = multiple_set(num_set = 10)
run_train_x, run_train_y = train_set(run_train, num_set = 10)

### testing data ###
run_test = pd.read_csv('test_runhist.csv').iloc[:, 2:]
run_test_x, run_test_y = label_divide(run_test, None, 'GB', train_only = True)
print('\n', 'Dimension of testing data:', run_test.shape)


Month 2:

Dimension of dataset 0 : (38022, 96)  balance ratio: 863.0
Dimension of dataset 1 : (1128, 96)  balance ratio: 1.0
Dimension of dataset 2 : (962, 96)  balance ratio: 1.0
Dimension of dataset 3 : (1226, 96)  balance ratio: 1.0
Dimension of dataset 4 : (1138, 96)  balance ratio: 1.0
Dimension of dataset 5 : (883, 96)  balance ratio: 1.0
Dimension of dataset 6 : (921, 96)  balance ratio: 1.0
Dimension of dataset 7 : (880, 96)  balance ratio: 1.0
Dimension of dataset 8 : (880, 96)  balance ratio: 1.0
Dimension of dataset 9 : (484, 96)  balance ratio: 10.0

 10 datasets are loaded.

Labels of  10 datasets are divided.

Month 3:

Dimension of dataset 0 : (60607, 107)  balance ratio: 550.0
Dimension of dataset 1 : (1800, 107)  balance ratio: 1.0
Dimension of dataset 2 : (2948, 107)  balance ratio: 1.0
Dimension of dataset 3 : (2030, 107)  balance ratio: 1.0
Dimension of dataset 4 : (1814, 107)  balance ratio: 1.0
Dimension of dataset 5 : (2203, 107)  balance ratio: 1.0
Dimension of